In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import emoji

In [13]:
dataframe = pd.read_csv("../texts.csv")
dataframe = dataframe.drop(["handle_id", "date", "message_date", "timestamp", "month", "year", "message_id", "chat_id", "phone_number"], axis=1)
dataframe = dataframe[2:]
mask = dataframe['text'].isin(['Cup Pong', 'Sea Battle', 'Chess', '8 Ball', '9 Ball', 'Archery', 'Paintball', 'Checkers', 'Basketball', 'Darts'])
dataframe = dataframe[~mask]
dataframe

,text,is_sent
2,You’re good you just said you’re gonna set it up,0
3,Text me lamo you have blue texts now,0
5,I’m downloading it,1
6,Lame,0
7,Very lame,0
...,...,...
37984,Enough,1
37985,But I wanna talk to you,0
37986,I got 92 on midterm btw,0
37987,Oh good job,1


In [14]:
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def clean(message):
    cleanedMessage = message.replace('\n',' ').lower()
    # Deal with some weird tokens
    cleanedMessage.replace("\xc2\xa0", "")
    cleanedMessage = cleanedMessage.replace("\xe2\x80\x99", "")
    #cleanedMessage.encode('ascii', 'ignore').decode('ascii')
    #cleanedMessage = cleanedMessage.replace(" u ", "  you ")
    # Remove punctuation
    cleanedMessage = give_emoji_free_text(cleanedMessage)
    cleanedMessage = re.sub('([.,!?])','', cleanedMessage)
    cleanedMessage = re.sub(' +', ' ', cleanedMessage) #remove extra spaces
    return cleanedMessage
print(str(clean("LOL 🤮🤮🤮 TEST")))

lol test


In [15]:
df = dataframe
df

,text,is_sent
2,You’re good you just said you’re gonna set it up,0
3,Text me lamo you have blue texts now,0
5,I’m downloading it,1
6,Lame,0
7,Very lame,0
...,...,...
37984,Enough,1
37985,But I wanna talk to you,0
37986,I got 92 on midterm btw,0
37987,Oh good job,1


In [ ]:
responseDictionary = dict()
prev = 0
firstspace = True
otherPersonsMessage, myMessage = "",""
for index,row in df.iterrows():
    #print(row['is_sent'], " ", row['text'])
    if (prev == 0):
        if (row['is_sent'] == 0): # 0 to 0
            otherPersonsMessage = str(otherPersonsMessage) + " " + str(row['text'])
            #print(otherPersonsMessage)
            prev = 0
        if (row['is_sent'] == 1): #0 to 1
            prev = 1
            firstspace = True
    if (prev == 1):
        if (row['is_sent'] == 1): # 1 to 1
            if (firstspace):
                myMessage += ((row['text']))
            else:
                myMessage += (" " + (row['text']))
            prev = 1
            firstspace = False
        if (row['is_sent'] == 0): #1 to 0
            #responseDictionary[clean((otherPersonsMessage.decode('utf8').encode('ascii', errors='ignore'))).rstrip()] = clean((myMessage.decode('utf8').encode('ascii', errors='ignore')).rstrip())
            print(clean(otherPersonsMessage).rstrip())
            print(clean(myMessage).rstrip())
            responseDictionary[clean(otherPersonsMessage).rstrip()] = clean(myMessage).rstrip()
            otherPersonsMessage = row['text']
            myMessage = ""
            prev = 0
            

In [21]:
combinedDictionary = {}
combinedDictionary.update(responseDictionary)
np.save('conversationDictionary.npy', combinedDictionary)
conversationFile = open('conversationData.txt', 'w')
for key, value in combinedDictionary.items():
    if (not key.strip() or not value.strip()):
        # If there are empty strings
        continue
    conversationFile.write(key.strip() + " " + value.strip() + " ")
print ("done")

done
